In [45]:
import os
import pandas as pd
from sqlalchemy import create_engine

In [47]:
# MySQL 접속 설정
username = "root"      # MySQL 사용자명
password = "byun1007"      # MySQL 비밀번호
host = "127.0.0.1"     # 로컬호스트
port = 3306            # 포트
database = "ott_db"  # DB 이름

# MySQL 엔진 생성
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

In [49]:
# 데이터 파일 경로 설정
folder_path = r"방송통신위원회_해외 OTT 이용행태조사 원시데이터_20231231"  # 경로의 띄어쓰기와 특수문자 확인
file_2022 = os.path.join(folder_path, "2022해외OTT이용행태조사_국가통합(20240403).csv")
file_2023 = os.path.join(folder_path, "2023해외OTT이용행태조사_국가통합(20240403).csv")

# CSV 읽기
df_2022 = pd.read_csv(file_2022, encoding='utf-8-sig', low_memory=False)
df_2023 = pd.read_csv(file_2023, encoding='utf-8-sig', low_memory=False)

print("2022 데이터:", df_2022.shape)
print("2023 데이터:", df_2023.shape)

2022 데이터: (4536, 1903)
2023 데이터: (6326, 929)


## 1. Payment
### user_id

In [51]:
# 1. 2022 user_id 생성
df_2022['user_id'] = ('2022' + df_2022['RESPID'].astype(str)).astype(int)
# 2. 2023 user_id 생성
df_2023['user_id'] = ('2023' + df_2023['id'].astype(str)).astype(int)

df_user_id_2022 = df_2022[['user_id']].copy()
df_user_id_2023 = df_2023[['user_id']].copy()

# 3. 합치기
df_user_id = pd.concat([df_user_id_2022, df_user_id_2023], ignore_index=True)

# 4. payment 테이블에 user_id 업로드
df_user_id.to_sql(name="payment", con=engine, index=False, if_exists="replace")

10862

### ott_cost_usa_2022, ott_cost_taiwan_2022, ott_cost_indonesia_2022

In [53]:
# 새로운 요금 매핑
cost_map = {
    99: '모르겠다',
    101: 'USD $10 미만', 102: 'USD $10-14', 103: 'USD $15-19', 104: 'USD $20 이상',
    201: 'TWD 200 미만', 202: 'TWD 200-299', 203: 'TWD 300-399', 204: 'TWD 400 이상',
    301: 'INR 50,000 미만', 302: 'INR 50,000-79,999', 303: 'INR 80,000-109,999',
    304: 'INR 110,000-139,999', 305: 'INR 140,000 이상'
}

# 미국
usa_codes = [99, 101, 102, 103, 104]
df_2022['ott_cost_usa_2022'] = df_2022['E2'].where(df_2022['E2'].isin(usa_codes)).map(cost_map)

# 대만
taiwan_codes = [99, 201, 202, 203, 204]
df_2022['ott_cost_taiwan_2022'] = df_2022['E2'].where(df_2022['E2'].isin(taiwan_codes)).map(cost_map)

# 인도네시아
indonesia_codes = [99, 301, 302, 303, 304, 305]
df_2022['ott_cost_indonesia_2022'] = df_2022['E2'].where(df_2022['E2'].isin(indonesia_codes)).map(cost_map)

In [55]:
# 1. 필요한 컬럼 추출
df_payment_2022 = df_2022[[
    'user_id', 
    'ott_cost_usa_2022', 
    'ott_cost_taiwan_2022', 
    'ott_cost_indonesia_2022'
]]

# 2. 기존 user_id와 병합
df_payment = pd.merge(
    df_user_id, df_payment_2022,
    on='user_id', how='left'
)

# 3. DB에 저장
df_payment.to_sql(name='payment', con=engine, index=False, if_exists='replace')

10862

### ott_cost_brazil_2023, ott_cost_mexico_2023, ott_cost_uk_2023, ott_cost_japan_2023

In [57]:
cost_map_2023 = {
    # 브라질 (1~5)
    1: "30 R$ 미만", 2: "30-59 R$", 3: "60-99 R$", 4: "100-199 R$", 5: "200 R$ 이상",

    # 멕시코 (21~25)
    21: "200 MXN 미만", 22: "200-399 MXN", 23: "400-599 MXN", 24: "600-999 MXN", 25: "1000 MXN 이상",

    # 영국 (31~36)
    31: "10 GBP 미만", 32: "10-19 GBP", 33: "20-29 GBP", 34: "30-39 GBP", 35: "40-59 GBP", 36: "60 GBP 이상",

    # 일본 (41~45)
    41: "1,000 JPY 미만", 42: "1,000-1,999 JPY", 43: "2,000-2,999 JPY",
    44: "3,000-4,999 JPY", 45: "5,000 JPY 이상"
}

# 브라질
brazil_codes = [1, 2, 3, 4, 5]
df_2023['ott_cost_brazil_2023'] = df_2023['D2'].where(df_2023['D2'].isin(brazil_codes)).map(cost_map_2023)

# 멕시코
mexico_codes = [21, 22, 23, 24, 25]
df_2023['ott_cost_mexico_2023'] = df_2023['D2'].where(df_2023['D2'].isin(mexico_codes)).map(cost_map_2023)

# 영국
uk_codes = [31, 32, 33, 34, 35, 36]
df_2023['ott_cost_uk_2023'] = df_2023['D2'].where(df_2023['D2'].isin(uk_codes)).map(cost_map_2023)

# 일본
japan_codes = [41, 42, 43, 44, 45]
df_2023['ott_cost_japan_2023'] = df_2023['D2'].where(df_2023['D2'].isin(japan_codes)).map(cost_map_2023)

In [59]:
# 1. 필요한 컬럼 추출
df_payment_2023 = df_2023[[
    'user_id',
    'ott_cost_brazil_2023',
    'ott_cost_mexico_2023',
    'ott_cost_uk_2023',
    'ott_cost_japan_2023'
]]

# 2. 기존 user_id와 병합
df_payment_final = pd.merge(
    df_payment,              # 기존 payment (2022 데이터 포함)
    df_payment_2023,         # 2023 국가 요금 데이터
    on='user_id', 
    how='outer'              # 2022, 2023을 모두 포함
)

# 3. DB에 저장
df_payment_final.to_sql(name='payment', con=engine, index=False, if_exists='replace')

10862

### pay_netflix_2022, pay_amazon_2022, pay_disney_2022

In [61]:
pay_map = {
    1: '본인이 전액 납부',
    2: '가족과 요금 분납',
    3: '지인과 요금 분납',
    4: '전액 납부된 타인의 계정 이용',
    5: '기타'
}

# 컬럼명 맞추기 (예시: 마침표가 들어가 있었던 경우)
df_2022['pay_netflix_2022'] = pd.to_numeric(df_2022['B3_1'], errors='coerce').map(pay_map)
df_2022['pay_amazon_2022']  = pd.to_numeric(df_2022['B3_2'], errors='coerce').map(pay_map)
df_2022['pay_disney_2022']  = pd.to_numeric(df_2022['B3_3'], errors='coerce').map(pay_map)

# 병합
df_payment_2022_pay = df_2022[[
    'user_id', 'pay_netflix_2022', 'pay_amazon_2022', 'pay_disney_2022'
]]

df_payment = pd.read_sql("SELECT * FROM payment", con=engine)
df_payment = pd.merge(df_payment, df_payment_2022_pay, on='user_id', how='left')
df_payment.to_sql(name='payment', con=engine, index=False, if_exists='replace')

10862

### pay_option1_2023, pay_option2_2023, pay_option3_2023, pay_option4_2023, pay_option5_2023

In [63]:
pay_map = {
    1: '본인이 전액 납부',
    2: '가족과 요금 분납',
    3: '지인과 요금 분납',
    4: '전액 납부된 타인의 계정 이용',
    5: '기타'
}

df_2023['pay_option1_2023'] = pd.to_numeric(df_2023['A3'], errors='coerce').map(pay_map)
df_2023['pay_option2_2023'] = pd.to_numeric(df_2023['A3_m2'], errors='coerce').map(pay_map)
df_2023['pay_option3_2023'] = pd.to_numeric(df_2023['A3_m3'], errors='coerce').map(pay_map)
df_2023['pay_option4_2023'] = pd.to_numeric(df_2023['A3_m4'], errors='coerce').map(pay_map)
df_2023['pay_option5_2023'] = pd.to_numeric(df_2023['A3_m5'], errors='coerce').map(pay_map)

df_payment_2023_pay = df_2023[[
    'user_id',
    'pay_option1_2023', 'pay_option2_2023', 'pay_option3_2023',
    'pay_option4_2023', 'pay_option5_2023'
]]

df_payment = pd.read_sql("SELECT * FROM payment", con=engine)
df_payment = pd.merge(df_payment, df_payment_2023_pay, on='user_id', how='left')
df_payment.to_sql(name='payment', con=engine, index=False, if_exists='replace')

10862